In [141]:
import cv2,os

In [142]:
data_path='dataset'

In [143]:
categories=os.listdir(data_path)

In [144]:
labels=[i for i in range(len(categories))]

In [145]:
label_dict=dict(zip(categories,labels))

In [146]:
print(label_dict)

{'with mask': 0, 'without mask': 1}


In [147]:
print(labels)

[0, 1]


In [148]:
print(categories)

['with mask', 'without mask']


In [149]:
img_size=100
data=[]
target=[]
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)
            

In [150]:
import np_utils

In [151]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils.np_utils import to_categorical

new_target=to_categorical(target)

In [152]:
np.save('data',data)
np.save('target',new_target)

In [153]:
target=new_target

In [154]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [155]:
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test = train_test_split(data, target, test_size=0.2)

In [156]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')


In [157]:
history=model.fit(X_train,X_test,epochs=5,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/5
28/28 [==============================] - 72s 3s/step - loss: 0.7502 - accuracy: 0.5330 - val_loss: 0.6764 - val_accuracy: 0.6955
Epoch 2/5
28/28 [==============================] - 48s 2s/step - loss: 0.5951 - accuracy: 0.7295 - val_loss: 0.4900 - val_accuracy: 0.7273
Epoch 3/5
28/28 [==============================] - 49s 2s/step - loss: 0.3886 - accuracy: 0.8352 - val_loss: 0.3178 - val_accuracy: 0.8682
Epoch 4/5
28/28 [==============================] - 48s 2s/step - loss: 0.2890 - accuracy: 0.8830 - val_loss: 0.2757 - val_accuracy: 0.8909
Epoch 5/5
28/28 [==============================] - 46s 2s/step - loss: 0.2213 - accuracy: 0.9216 - val_loss: 0.3625 - val_accuracy: 0.8591


In [1]:
import matplotlib.pyplot as plt
from keras.models import load_model
%matplotlib inline

In [ ]:
import cv2
import numpy as np
model = load_model('model-017.model')

In [10]:
face_cls=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
source=cv2.VideoCapture(0)
labels_dict={0:' MASK ',1:' NO MASK!!! '}
color_dict={0:(0,255,0),1:(0,0,255)}

In [11]:
while(True):
    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cls.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    if(key==27):
        break
cv2.destroyAllWindows()
source.release()

